### **1. Paramètres**

In [1]:
# Importation des bibliothèques
import os
import pandas as pd

In [2]:
# Chemin vers les fichiers
csv_path = '../data_final/tableaux/'

# Lire le fichier CSV
df_pixel = pd.read_csv(os.path.join(csv_path, 'df_pixel.csv'), dtype={'id_parc': str})
df_parcelle = pd.read_csv(os.path.join(csv_path, 'df_parcelle.csv'), dtype={'id_parc': str})

### **2. Fonction pour filtrer les .csv**

In [3]:
def filtrer_dataframe(df, pixels_to_exclude, exclude_unique_ids, criteria, lidar_date_filter=False):
    """
    Applique des filtres au DataFrame pour exclure les pixels selon des critères spécifiques.

    Args:
        df (DataFrame): Le DataFrame à filtrer.
        pixels_to_exclude (list): Liste des coordonnées (x, y) à exclure.
        exclude_unique_ids (list): Liste des `unique_id` à exclure complètement.
        criteria (dict): Critères spécifiques pour certains `unique_id`.
        lidar_date_filter (bool): Si True, filtre également selon 'lidar_date' == 'date' et autres conditions Lidar.

    Returns:
        DataFrame: Le DataFrame filtré.
    """
    # Supprimer les pixels selon les coordonnées
    df_filtre = df[~df[["x", "y"]].apply(tuple, axis=1).isin(pixels_to_exclude)]
    
    # Supprimer les pixels selon `unique_id`
    df_filtre = df_filtre[~df_filtre["unique_id"].isin(exclude_unique_ids)]
    
    # Appliquer les critères spécifiques
    for uid, condition in criteria.items():
        df_filtre = df_filtre[~(
            (df_filtre["unique_id"] == uid) &
            (df_filtre.apply(condition, axis=1))
        )]

    # Filtrer selon 'lidar_date' == 'date' et autres conditions spécifiques à Lidar
    if lidar_date_filter:
        # Condition 1: 'lidar_date' == 'date'
        if 'lidar_date' in df_filtre.columns and 'date' in df_filtre.columns:
            df_filtre = df_filtre[df_filtre['lidar_date'] == df_filtre['date']]
        
        # Condition 2: Supprimer où `grid_CC < 5` et `age_plan >= 5`
        if 'grid_CC' in df_filtre.columns and 'age_plan' in df_filtre.columns:
            df_filtre = df_filtre[~((df_filtre['grid_CC'] < 5) & (df_filtre['age_plan'] >= 5))]
        
        # FIltrage dans l'échelle parcelle en utilisant la moyenne du CC (`CC`)
        # if 'CC' in df_filtre.columns and 'age_plan' in df_filtre.columns:
        #     df_filtre = df_filtre[~((df_filtre['CC'] < 5) & (df_filtre['age_plan'] >= 5))]

    return df_filtre


### **3. Filtrage des pixels du Departément 10**

In [4]:
# Définir les paramètres de filtrage
pixels_to_exclude = [
    (736105, 6824467.46), (736115, 6824467.46), (736125, 6824467.46),
    (736785, 6824337.46), (736795, 6824337.46), (736805, 6824337.46)
]

exclude_unique_ids = [
    "dep10_4", "dep10_5", "dep10_6", "dep10_8", "dep10_9", "dep10_14",
    "dep10_17", "dep10_18", "dep10_19", "dep10_20", "dep10_21", "dep10_24",
    "dep10_27", "dep10_29", "dep10_33", "dep10_35", "dep10_38", "dep10_40",
    "dep10_41", "dep10_42", "dep10_45", "dep10_48", "dep10_49", "dep10_50",
    "dep10_51", "dep10_52", "dep10_57", "dep10_59", "dep10_69", "dep10_71",
    "dep10_83", "dep10_87"
]

criteria = {
    "dep10_15": lambda row: row['date'] <= 2019,
    "dep10_65": lambda row: row['date'] <= 2019,
    "dep10_66": lambda row: row['date'] <= 2019,
    "dep10_77": lambda row: row['date'] <= 2019
}


### **3.1 Filtrage des pixels dans le .csv issue des raster Lidar (échelle pixel)**

In [5]:
# Avant le filtrage - statistiques initiales
print("\n*** Statistiques avant le filtrage (pixels, dept10) ***")
print(f"Quantité totale de lignes : {len(df_pixel)}")  # Nombre total de lignes
print(f"Quantité totale de unique_ids : {df_pixel['unique_id'].nunique()}")  # Nombre total de unique_ids
print("Quantité de lignes par département (source) :")
print(df_pixel.groupby('source').size())  # Nombre de lignes par département
print("Quantité de unique_ids par département (source) :")
print(df_pixel.groupby('source')['unique_id'].nunique())  # Nombre de unique_ids par département

# Appliquer le filtrage
df_pixel_filtre_dept10 = filtrer_dataframe(df_pixel, pixels_to_exclude, exclude_unique_ids, criteria)

# Après le filtrage - statistiques finales
print("\n*** Statistiques après le filtrage (pixels, dept10) ***")
print(f"Quantité totale de lignes : {len(df_pixel_filtre_dept10)}")  # Nombre total de lignes restantes
print(f"Quantité totale de unique_ids : {df_pixel_filtre_dept10['unique_id'].nunique()}")  # Nombre total de unique_ids restants
print("Quantité de lignes par département (source) :")
print(df_pixel_filtre_dept10.groupby('source').size())  # Nombre de lignes restantes par département
print("Quantité de unique_ids par département (source) :")
print(df_pixel_filtre_dept10.groupby('source')['unique_id'].nunique())  # Nombre de unique_ids restants par département



*** Statistiques avant le filtrage (pixels, dept10) ***
Quantité totale de lignes : 77617
Quantité totale de unique_ids : 323
Quantité de lignes par département (source) :
source
dep10       15199
dep47       18061
dep73       22110
dep82_bb    10654
dep82_sp    11593
dtype: int64
Quantité de unique_ids par département (source) :
source
dep10        64
dep47        60
dep73        19
dep82_bb     42
dep82_sp    138
Name: unique_id, dtype: int64

*** Statistiques après le filtrage (pixels, dept10) ***
Quantité totale de lignes : 71453
Quantité totale de unique_ids : 298
Quantité de lignes par département (source) :
source
dep10        9035
dep47       18061
dep73       22110
dep82_bb    10654
dep82_sp    11593
dtype: int64
Quantité de unique_ids par département (source) :
source
dep10        39
dep47        60
dep73        19
dep82_bb     42
dep82_sp    138
Name: unique_id, dtype: int64


In [6]:
# Sauvegarder le DataFrame filtré
df_pixel_filtre_dept10.to_csv(os.path.join(csv_path, 'df_pixel_filtre_dept10.csv'), index=False)
print(f"Fichier df_px_filtre_dept10 sauvegardé avec succès.")

Fichier df_px_filtre_dept10 sauvegardé avec succès.


### **3.2 Filtrage des pixels dans le .csv issue des métriques calculées pour les parcelles (échelle parcelle)**

In [7]:
# Avant le filtrage - statistiques initiales
print("\n*** Statistiques avant le filtrage (parcelles, dept10) ***")
print(f"Quantité totale de lignes : {len(df_parcelle)}")  # Nombre total de lignes
print(f"Quantité totale de unique_ids : {df_parcelle['unique_id'].nunique()}")  # Nombre total de unique_ids
print("Quantité de lignes par département (source) :")
print(df_parcelle.groupby('source').size())  # Nombre de lignes par département
print("Quantité de unique_ids par département (source) :")
print(df_parcelle.groupby('source')['unique_id'].nunique())  # Nombre de unique_ids par département

# Appliquer le filtrage
df_parcelle_filtre_dept10 = filtrer_dataframe(df_parcelle, pixels_to_exclude, exclude_unique_ids, criteria)

# Après le filtrage - statistiques finales
print("\n*** Statistiques après le filtrage (parcelles, dept10) ***")
print(f"Quantité totale de lignes : {len(df_parcelle_filtre_dept10)}")  # Nombre total de lignes restantes
print(f"Quantité totale de unique_ids : {df_parcelle_filtre_dept10['unique_id'].nunique()}")  # Nombre total de unique_ids restants
print("Quantité de lignes par département (source) :")
print(df_parcelle_filtre_dept10.groupby('source').size())  # Nombre de lignes restantes par département
print("Quantité de unique_ids par département (source) :")
print(df_parcelle_filtre_dept10.groupby('source')['unique_id'].nunique())  # Nombre de unique_ids restants par département



*** Statistiques avant le filtrage (parcelles, dept10) ***
Quantité totale de lignes : 77617
Quantité totale de unique_ids : 323
Quantité de lignes par département (source) :
source
dep10       15199
dep47       18061
dep73       22110
dep82_bb    10654
dep82_sp    11593
dtype: int64
Quantité de unique_ids par département (source) :
source
dep10        64
dep47        60
dep73        19
dep82_bb     42
dep82_sp    138
Name: unique_id, dtype: int64

*** Statistiques après le filtrage (parcelles, dept10) ***
Quantité totale de lignes : 71453
Quantité totale de unique_ids : 298
Quantité de lignes par département (source) :
source
dep10        9035
dep47       18061
dep73       22110
dep82_bb    10654
dep82_sp    11593
dtype: int64
Quantité de unique_ids par département (source) :
source
dep10        39
dep47        60
dep73        19
dep82_bb     42
dep82_sp    138
Name: unique_id, dtype: int64


In [8]:
# Sauvegarder le DataFrame filtré
df_parcelle_filtre_dept10.to_csv(os.path.join(csv_path, 'df_parcelle_filtre_dept10.csv'), index=False)
print(f"Fichier df_parcelle_filtre_dept10 sauvegardé avec succès.")

Fichier df_parcelle_filtre_dept10 sauvegardé avec succès.


### **4. Filtrage pour les métriques lidar**

### **4.1 Filtrage pour les métriques lidar dans le .csv issue des raster Lidar (échelle pixel)**

In [9]:
# Avant le filtrage lidar - statistiques initiales
print("\n*** Statistiques avant le filtrage lidar (pixels) ***")
print(f"Quantité totale de lignes : {len(df_pixel)}")  # Nombre total de lignes
print(f"Quantité totale de unique_ids : {df_pixel['unique_id'].nunique()}")  # Nombre total de unique_ids
print("Quantité de lignes par département (source) :")
print(df_pixel.groupby('source').size())  # Nombre de lignes par département
print("Quantité de unique_ids par département (source) :")
print(df_pixel.groupby('source')['unique_id'].nunique())  # Nombre de unique_ids par département

# Appliquer le filtrage lidar
df_pixel_filtre_lidar = filtrer_dataframe(df_pixel, pixels_to_exclude, exclude_unique_ids, criteria, lidar_date_filter=True)

# Après le filtrage lidar - statistiques finales
print("\n*** Statistiques après le filtrage lidar (pixels) ***")
print(f"Quantité totale de lignes : {len(df_pixel_filtre_lidar)}")  # Nombre total de lignes restantes
print(f"Quantité totale de unique_ids : {df_pixel_filtre_lidar['unique_id'].nunique()}")  # Nombre total de unique_ids restants
print("Quantité de lignes par département (source) :")
print(df_pixel_filtre_lidar.groupby('source').size())  # Nombre de lignes restantes par département
print("Quantité de unique_ids par département (source) :")
print(df_pixel_filtre_lidar.groupby('source')['unique_id'].nunique())  # Nombre de unique_ids restants par département



*** Statistiques avant le filtrage lidar (pixels) ***
Quantité totale de lignes : 77617
Quantité totale de unique_ids : 323
Quantité de lignes par département (source) :
source
dep10       15199
dep47       18061
dep73       22110
dep82_bb    10654
dep82_sp    11593
dtype: int64
Quantité de unique_ids par département (source) :
source
dep10        64
dep47        60
dep73        19
dep82_bb     42
dep82_sp    138
Name: unique_id, dtype: int64

*** Statistiques après le filtrage lidar (pixels) ***
Quantité totale de lignes : 9031
Quantité totale de unique_ids : 198
Quantité de lignes par département (source) :
source
dep47       1913
dep73       3747
dep82_bb     732
dep82_sp    2639
dtype: int64
Quantité de unique_ids par département (source) :
source
dep47        25
dep73        14
dep82_bb     32
dep82_sp    127
Name: unique_id, dtype: int64


In [10]:
# Sauvegarder le DataFrame filtré
df_pixel_filtre_lidar.to_csv(os.path.join(csv_path, 'df_pixel_filtre_lidar.csv'), index=False)
print(f"Fichier df_pixel_filtre_lidar sauvegardé avec succès.")

Fichier df_pixel_filtre_lidar sauvegardé avec succès.


### **4.2 Filtrage pour les métriques lidar dans le .csv issue des métriques calculées pour les parcelles (échelle parcelle)**

In [11]:
# Avant le filtrage lidar - statistiques initiales
print("\n*** Statistiques avant le filtrage lidar (parcelles) ***")
print(f"Quantité totale de lignes : {len(df_parcelle)}")  # Nombre total de lignes
print(f"Quantité totale de unique_ids : {df_parcelle['unique_id'].nunique()}")  # Nombre total de unique_ids
print("Quantité de lignes par département (source) :")
print(df_parcelle.groupby('source').size())  # Nombre de lignes par département
print("Quantité de unique_ids par département (source) :")
print(df_parcelle.groupby('source')['unique_id'].nunique())  # Nombre de unique_ids par département

# Appliquer le filtrage lidar
df_parcelle_filtre_lidar = filtrer_dataframe(df_parcelle, pixels_to_exclude, exclude_unique_ids, criteria, lidar_date_filter=True)

# Après le filtrage lidar - statistiques finales
print("\n*** Statistiques après le filtrage lidar (parcelles) ***")
print(f"Quantité totale de lignes : {len(df_parcelle_filtre_lidar)}")  # Nombre total de lignes restantes
print(f"Quantité totale de unique_ids : {df_parcelle_filtre_lidar['unique_id'].nunique()}")  # Nombre total de unique_ids restants
print("Quantité de lignes par département (source) :")
print(df_parcelle_filtre_lidar.groupby('source').size())  # Nombre de lignes restantes par département
print("Quantité de unique_ids par département (source) :")
print(df_parcelle_filtre_lidar.groupby('source')['unique_id'].nunique())  # Nombre de unique_ids restants par département



*** Statistiques avant le filtrage lidar (parcelles) ***
Quantité totale de lignes : 77617
Quantité totale de unique_ids : 323
Quantité de lignes par département (source) :
source
dep10       15199
dep47       18061
dep73       22110
dep82_bb    10654
dep82_sp    11593
dtype: int64
Quantité de unique_ids par département (source) :
source
dep10        64
dep47        60
dep73        19
dep82_bb     42
dep82_sp    138
Name: unique_id, dtype: int64

*** Statistiques après le filtrage lidar (parcelles) ***
Quantité totale de lignes : 10111
Quantité totale de unique_ids : 207
Quantité de lignes par département (source) :
source
dep47       1918
dep73       3772
dep82_bb    1724
dep82_sp    2697
dtype: int64
Quantité de unique_ids par département (source) :
source
dep47        25
dep73        14
dep82_bb     39
dep82_sp    129
Name: unique_id, dtype: int64


In [12]:
# Sauvegarder le DataFrame filtré
df_parcelle_filtre_lidar.to_csv(os.path.join(csv_path, 'df_parcelle_filtre_lidar.csv'), index=False)
print(f"Fichier df_parcelle_filtre_lidar sauvegardé avec succès.")

Fichier df_parcelle_filtre_lidar sauvegardé avec succès.


## **5. Supprimer le département 10** 

In [13]:
# Avant la suppression du département 10 - statistiques initiales
print("\n*** Statistiques avant la suppression du département 10 ***")
print(f"Quantité totale de lignes : {len(df_pixel)}")  # Nombre total de lignes
print(f"Quantité totale de unique_ids : {df_pixel['unique_id'].nunique()}")  # Nombre total de unique_ids
print("Quantité de lignes par département (source) :")
print(df_pixel.groupby('source').size())  # Nombre de lignes par département
print("Quantité de unique_ids par département (source) :")
print(df_pixel.groupby('source')['unique_id'].nunique())  # Nombre de unique_ids par département

# Supprimer les données du département 10
df_px_filtre_sans_dept10 = df_pixel[~(df_pixel['source'] == 'dep10')]

# Après la suppression du département 10 - statistiques finales
print("\n*** Statistiques après la suppression du département 10 ***")
print(f"Quantité totale de lignes : {len(df_px_filtre_sans_dept10)}")  # Nombre total de lignes restantes
print(f"Quantité totale de unique_ids : {df_px_filtre_sans_dept10['unique_id'].nunique()}")  # Nombre total de unique_ids restants
print("Quantité de lignes par département (source) :")
print(df_px_filtre_sans_dept10.groupby('source').size())  # Nombre de lignes restantes par département
print("Quantité de unique_ids par département (source) :")
print(df_px_filtre_sans_dept10.groupby('source')['unique_id'].nunique())  # Nombre de unique_ids restants par département



*** Statistiques avant la suppression du département 10 ***
Quantité totale de lignes : 77617
Quantité totale de unique_ids : 323
Quantité de lignes par département (source) :
source
dep10       15199
dep47       18061
dep73       22110
dep82_bb    10654
dep82_sp    11593
dtype: int64
Quantité de unique_ids par département (source) :
source
dep10        64
dep47        60
dep73        19
dep82_bb     42
dep82_sp    138
Name: unique_id, dtype: int64

*** Statistiques après la suppression du département 10 ***
Quantité totale de lignes : 62418
Quantité totale de unique_ids : 259
Quantité de lignes par département (source) :
source
dep47       18061
dep73       22110
dep82_bb    10654
dep82_sp    11593
dtype: int64
Quantité de unique_ids par département (source) :
source
dep47        60
dep73        19
dep82_bb     42
dep82_sp    138
Name: unique_id, dtype: int64


In [14]:
# Sauvegarder le DataFrame filtré
df_px_filtre_sans_dept10.to_csv(os.path.join(csv_path, 'df_pixel_sans_dept10.csv'), index=False)
print(f"Fichier df_pixel_sans_dept10 sauvegardé avec succès.")

Fichier df_pixel_sans_dept10 sauvegardé avec succès.
